In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import plotly.express as px
import math

In [ ]:
data = np.load('Era_wind.npz')

In [ ]:
list(data.keys())

In [ ]:
for key in data.keys():
    print(key)                       
    print(data[key])  

In [ ]:
wind_v = data["wind_v"][:1, 3]

In [ ]:
print(wind_v.shape)

In [ ]:
print(wind_v)

In [ ]:
level_index = 3 # get all values at 300 Pa
lat_index = 0
lon_index = 0

wind_v_subset = data['wind_v'][:24, level_index, lat_index, lon_index]

In [ ]:
lat = data['wind_v'][:1, 3, 1, 1]
print(lat)

In [ ]:
lons = data['lon']
lats = data['lat']

print("Longitude range:", np.ptp(lons))
print("Latitude range:", np.ptp(lats))

In [ ]:
wide = np.squeeze(wind_v)

In [ ]:
new_wind_v.shape

In [ ]:
first_hour = pd.DataFrame(wide)
first_hour.columns = lons
first_hour.index = lats

In [ ]:
first_hour

In [ ]:
first_hourm = first_hour.melt(ignore_index=False).reset_index()


In [ ]:
first_hourm.columns = ['lat', 'lon', 'wind_v']
first_hourm

In [ ]:
wind_u = data["wind_u"][:1, 3]

In [ ]:
wide_u = np.squeeze(wind_u)

In [ ]:
wide_u.shape

In [ ]:
first_hour_u = pd.DataFrame(wide_u)
first_hour_u.columns = lons
first_hour_u.index = lats

In [ ]:
first_hour_u

In [ ]:
first_hour_um = first_hour_u.melt(ignore_index=False).reset_index()


In [ ]:
first_hour_um.columns = ['lat', 'lon', 'wind_u']
first_hour_um

In [ ]:
first_hour_comb = pd.merge(first_hourm, first_hour_um, 'left', on = ['lat', 'lon'])

In [ ]:
first_hour_comb

In [ ]:
first_hour_comb = first_hour_comb.assign(wind_scalar = lambda first_hour_comb: np.sqrt(first_hour_comb.wind_v**2 + first_hour_comb.wind_u**2)) 

In [ ]:
first_hour_comb

In [ ]:
fig = px.scatter_geo(first_hour_comb,lat='lat',lon='lon', hover_name="wind_scalar", color = "wind_scalar")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

Create function that creates a dataframe of each component of wind at 300 Pa and then joins them and calculates wind scalar

In [ ]:
def wind_scalar(hour):
    
    lons = data['lon']
    lats = data['lat']

    wind_v = data['wind_v'][hour-1:hour, 3] # get values at all lat and long at 300 Pa for one hour
    
    wind_v_squeeze = np.squeeze(wind_v) # turn 3d array into 2d (collapse into one table)
    
    wind_v_df = pd.DataFrame(wind_v_squeeze) # convert 2d array to DF
    wind_v_df.columns = lons           # change cols to longitude values
    wind_v_df.index = lats             # change rows to lat values
    
    wind_v_melt = wind_v_df.melt(ignore_index=False).reset_index() # pivot the df longer so each lat and long has its own row
    wind_v_melt.columns = ['lat', 'lon', 'wind_v']
    
    # repeat process for the u value
    wind_u = data['wind_u'][hour-1:hour, 3] # get values at all lat and long at 300 Pa for one hour
    
    wind_u_squeeze = np.squeeze(wind_u) # turn 3d array into 2d (collapse into one table)
    
    wind_u_df = pd.DataFrame(wind_u_squeeze) # convert 2d array to DF
    wind_u_df.columns = lons           # change cols to longitude values
    wind_u_df.index = lats             # change rows to lat values
    
    wind_u_melt = wind_u_df.melt(ignore_index=False).reset_index() # pivot the df longer so each lat and long has its own row
    wind_u_melt.columns = ['lat', 'lon', 'wind_u']
    
    
    # join the two dfs 
    combined = pd.merge(wind_v_melt, wind_u_melt, 'left', on = ['lat', 'lon'])
    
    # create new scalar column
    combined = combined.assign(wind_scalar = lambda combined: np.sqrt(combined.wind_v**2 + combined.wind_u**2)) 
    combined['hour'] = hour
    
    return combined


In [ ]:
hour2 = wind_scalar(2)

In [ ]:
hour2

In [ ]:
result = []

for hour in range(1,25):
    df_loop = wind_scalar(hour)
    
    result.append(df_loop)


In [ ]:
final = pd.concat(result)

In [ ]:
fig = px.scatter_geo(final,lat='lat',lon='lon', hover_name="wind_scalar", color = "wind_scalar", animation_frame = 'hour')
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [ ]:
hour2 = wind_scalar(23)

fig = px.scatter_geo(hour2,lat='lat',lon='lon', hover_name="wind_scalar", color = "wind_scalar")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [ ]:
hour3 = wind_scalar(1)

fig = px.scatter_geo(hour3,lat='lat',lon='lon', hover_name="wind_scalar", color = "wind_scalar")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [ ]:
hour3